In [1]:
import h5py
import simulation2 as sim
import numpy as np
import pandas as pd

def generate_h5(location, name):
    catalog = h5py.File(location, 'r')
    
    dID = catalog['ID']
    dM200 = catalog['M200']
    dM500 = catalog['M500']
    dNoise = catalog['Noise Map']
    dR200 = catalog['R200']
    dRedshift = catalog['Redshift']
    dTempSubmap = catalog['Temperature Submap']
    dc200 = catalog['c200']
    
    # Store data in numpy arrays
    dataID = dID[:]
    dataM200 = dM200[:]
    dataM500 = dM500[:]
    dataNoise = dNoise[:]
    dataR200 = dR200[:]
    dataRedshift = dRedshift[:]
    dataTempSubmap = dTempSubmap[:]
    datac200 = dc200[:]
    
    omega_m0 = catalog.attrs['omega_m0']
    omega_b0 = catalog.attrs['omega_b0']
    cosmo_h = catalog.attrs['cosmo_h']
    sigma8 = catalog.attrs['sigma8']
    ns = catalog.attrs['ns']
    tele = catalog.attrs['telescope']
    f = catalog.attrs['frequency']
    beam_size = catalog.attrs['Beam Size']
    noise_level = catalog.attrs['Noise Level']
    rad = catalog.attrs['Random Seed']
    filepath = catalog.attrs['Catalog']
    
    # Create Compton-y map with noise
    t_cmb = 2.725 
    f = 150
    fsz = sim.f_sz(f, t_cmb)
    y_noise = dataTempSubmap / fsz / t_cmb / 1e6
    
    # Extract SZ signal and determine aperture size
    dataSignal = np.array([])
    dataAperture = np.array([])

    for i in range (len(dataRedshift)):
        SZsignal, aperture = sim.tSZ_signal(dataRedshift[i], y_noise[i])
        dataSignal = np.append(dataSignal, SZsignal)
        dataAperture = np.append(dataAperture, aperture)
    
    saveSignal = dataSignal
    saveAperture = dataAperture
    
    # Eliminate nan data
    for i in range (len(dataRedshift)):
        if pd.isna(saveSignal[i]):
            dataSignal = saveSignal[i+1:]
        else:
            break
    dataID = dataID[i:]
    dataM200 = dataM200[i:]
    dataM500 = dataM500[i:]
    dataNoise = dataNoise[i:]
    dataR200 = dataR200[i:]
    dataRedshift = dataRedshift[i:]
    dataTempSubmap = dataTempSubmap[i:]
    datac200 = datac200[i:]
    dataAperture = dataAperture[i:]
    
    # Create the new h5 file
    f1 = h5py.File(name, "w")
    dset1 = f1.create_dataset("ID", data=dataID)
    dset2 = f1.create_dataset("M200", data=dataM200)
    dset3 = f1.create_dataset("M500", data=dataM500)
    dset4 = f1.create_dataset("Noise Map", data=dataNoise)
    dset5 = f1.create_dataset("R200", data=dataR200)
    dset6 = f1.create_dataset("Temperature Submap", data=dataTempSubmap)
    dset7 = f1.create_dataset("c200", data=datac200)
    dset8 = f1.create_dataset("SZ signal", data=dataSignal)
    dset9 = f1.create_dataset("Aperture Size", data=dataAperture)
    dset10 = f1.create_dataset("Redshift", data=dataRedshift)
    
    f1.attrs["omega_m0"] = omega_m0
    f1.attrs["omega_b0"] = omega_b0
    f1.attrs["cosmo_h"] = cosmo_h
    f1.attrs["sigma8"] = sigma8
    f1.attrs["ns"] = ns
    f1.attrs["telescope"] = tele
    f1.attrs["frequency"] = f
    f1.attrs["Beam Size"] = beam_size
    f1.attrs["Noise Level"] = noise_level
    f1.attrs["Random Seed"] = rad
    f1.attrs["Catalog"] = filepath
    
    f1.close()

In [2]:
catalog1location = "/Users/cristescuioana/Desktop/GalaxyClusters/Catalog/2022-08-05-N1.hdf5"
name1 = "2022-08-05-N1_new.hdf5"
generate_h5(catalog1location, name1)

/Users/cristescuioana/Desktop/GalaxyClusters/Catalog/simulation2.py:229: RuntimeWarning: Mean of empty slice.
  ring_mean = Map[(r >= rin) & (r < rout)].mean()
/Users/cristescuioana/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
catalog2location = "/Users/cristescuioana/Desktop/GalaxyClusters/Catalog/2022-08-05-N2.hdf5"
name2 = "2022-08-05-N2_new.hdf5"
generate_h5(catalog2location, name2)